Imports
====

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load Data
=====

In [18]:
new_data = pd.read_csv('store.csv')
data = pd.read_csv('postProcessed.csv')
joined = data.merge(new_data, how='outer', on='Store')

In [34]:
joined.dtypes

Store                          int64
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
DateTime                      object
Year                           int64
Month                          int64
Day                            int64
WeekNumber                     int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [40]:
joined.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,Year,Month,Day,WeekNumber,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1017209.000000,1014567.000000,693861.000000,693861.000000,1017209.000000,1017209.000000,1017209.000000
mean,558.429727,3.998341,5773.818972,633.145946,0.830107,0.381515,0.178647,2013.832292,5.846762,15.702790,23.615515,5430.085652,7.222866,2008.690228,0.500564,11.647665,1007.010608
std,321.908651,1.997391,3849.926175,464.411734,0.375539,0.485759,0.383056,0.777396,3.326097,8.787638,14.433381,7715.323700,3.211832,5.992644,0.500000,15.323928,1005.876930
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000,1.000000,1.000000,1.000000,20.000000,1.000000,1900.000000,0.000000,0.000000,0.000000
25%,280.000000,2.000000,3727.000000,405.000000,1.000000,0.000000,0.000000,2013.000000,3.000000,8.000000,11.000000,710.000000,4.000000,2006.000000,0.000000,0.000000,0.000000
50%,558.000000,4.000000,5744.000000,609.000000,1.000000,0.000000,0.000000,2014.000000,6.000000,16.000000,22.000000,2330.000000,8.000000,2010.000000,1.000000,1.000000,2009.000000
75%,838.000000,6.000000,7856.000000,837.000000,1.000000,1.000000,0.000000,2014.000000,8.000000,23.000000,35.000000,6890.000000,10.000000,2013.000000,1.000000,22.000000,2012.000000
max,1115.000000,7.000000,41551.000000,7388.000000,1.000000,1.000000,1.000000,2015.000000,12.000000,31.000000,52.000000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


In [39]:
joined.ix[:, 'StateHoliday'] = joined.ix[:, 'StateHoliday'].astype(str)
joined.ix[:, ['Promo2SinceWeek', 
              'Promo2SinceYear', 
              'PromoInterval']] = joined.ix[:, ['Promo2SinceWeek', 
                                                'Promo2SinceYear', 
                                                'PromoInterval']].fillna(0)

Need to fix all of the categorical variables
========================

In [33]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
enc.fit(np.unique(joined['StateHoliday']))


enc.transform(joined['StateHoliday'])

array([0, 1, 2, 3])

In [71]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()
type_assort = dv.fit_transform(joined[['StoreType', 'Assortment']].to_dict(orient='records')).toarray()


In [81]:
joined['TypeAssort'] = pd.Series(list(type_assort))


In [98]:
from sklearn.cross_validation import train_test_split

response = joined['Sales']

columns = ['Store', 
           'DayOfWeek', 
           'Customers', 
           'Open', 
           'Promo', 
           'StateHoliday', 
           'SchoolHoliday',
           'Year', 
           'Month', 
           'Day', 
           'WeekNumber',
           'CompetitionDistance', 
           'CompetitionOpenSinceMonth', 
           'CompetitionOpenSinceYear', 
           'Promo2', 
           'Promo2SinceWeek', 
           'Promo2SinceYear']

X_train, X_test, y_train, y_test = train_test_split(joined.ix[:,columns], response, test_size=0.25, random_state=8675309)

In [99]:
from sklearn.tree import DecisionTreeRegressor

explore_tree = DecisionTreeRegressor().fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [95]:
X_train.describe()

,Store,DayOfWeek,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekNumber,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,762906.000000,760952.000000,520514.000000,520514.000000,762906.000000,381897.000000,381897.000000
mean,558.408179,3.999929,633.007827,0.829873,0.381410,0.030465,0.178908,2013.831834,5.844982,15.703186,23.608801,5430.967012,7.225352,2008.686779,0.500582,23.252903,2011.752551
std,321.842033,1.997154,464.407844,0.375745,0.485733,0.171863,0.383276,0.777144,3.326522,8.789229,14.435145,7715.248923,3.211848,6.011545,0.500000,14.098609,1.662586
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000,1.000000,1.000000,1.000000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,280.000000,2.000000,405.000000,1.000000,0.000000,0.000000,0.000000,2013.000000,3.000000,8.000000,11.000000,710.000000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.000000,4.000000,609.000000,1.000000,0.000000,0.000000,0.000000,2014.000000,6.000000,16.000000,22.000000,2330.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,837.000000,6.000000,837.000000,1.000000,1.000000,0.000000,0.000000,2014.000000,8.000000,23.000000,35.000000,6890.000000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.000000,7.000000,7388.000000,1.000000,1.000000,1.000000,1.000000,2015.000000,12.000000,31.000000,52.000000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000
